In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
all_sheets = pd.read_excel('Raw Data/SCB_act_dbie.xlsx', sheet_name=None)
ms = pd.read_excel('Raw Data/money_stock.xlsx')
cred_deploy = pd.read_excel('Raw Data/cred_deploy.xlsx')
print("Sheet Names:", all_sheets.keys())

/opt/anaconda3/envs/Py_3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Sheet Names: dict_keys(['Major Balance Sheet Items', 'Capital Adequacy', 'Assest Quality', 'Profitability', 'Performance Indicators', 'Major Sectoral', 'Priority-Non_Priority Break-up', 'Infrastructure Financing Items'])


/opt/anaconda3/envs/Py_3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
all_sheets_2 = pd.read_excel('Raw Data/SCB_act_dbie_2.xlsx', sheet_name=None)
cred_deploy_2 = pd.read_excel('Raw Data/cred_deploy_2.xlsx')
print("Sheet Names:", all_sheets_2.keys())

/opt/anaconda3/envs/Py_3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Sheet Names: dict_keys(['Major Balance Sheet Items', 'Capital Adequacy', 'Assest Quality', 'Profitability', 'Performance Indicators', 'Major Sectoral', 'Priority-Non_Priority Break-up', 'Infrastructure Financing Items'])


In [4]:
Foreign_Banks = ["AB BANK LIMITED", "ABU DHABI COMMERCIAL BANK PJSC" ,"AMERICAN EXPRESS BANKING CORP.", "AUSTRALIA AND NEW ZEALAND BANKING GROUP LIMITED" ,"BANK OF AMERICA , NATIONAL ASSOCIATION" ,"BANK OF BAHRAIN & KUWAIT B.S.C." ,"BANK OF CEYLON", "BANK OF NOVA SCOTIA", "BARCLAYS BANK PLC" ,"BNP PARIBAS" ,"CITIBANK N.A" ,"COMMONWEALTH BANK OF AUSTRALIA", "COOPERATIEVE RABOBANK U.A." ,"CREDIT AGRICOLE CORPORATE AND INVESTMENT BANK", "CREDIT SUISSE AG", "CTBC BANK CO., LTD." ,"DBS BANK INDIA LTD.", "DEUTSCHE BANK AG" ,"FIRSTRAND BANK LTD" ,"HONGKONG AND SHANGHAI BANKING CORPN.LTD." ,"HSBC BANK OMAN S.A.O.G." ,"INDUSTRIAL AND COMMERCIAL BANK OF CHINA", "JPMORGAN CHASE BANK NATIONAL ASSOCIATION", "JSC VTB BANK", "KBC BANK NV" ,"KRUNG THAI BANK PUBLIC COMPANY LIMITED" ,"MASHREQ BANK PSC", "MIZUHO BANK LTD", "MUFG Bank Ltd", "NATIONAL AUSTRALIA BANK", "NatWest Markets Plc", "SBERBANK" ,"SBM BANK (INDIA) LTD." ,"SHINHAN BANK" ,"SOCIETE GENERALE" ,"SONALI BANK", "STANDARD CHARTERED BANK" ,"UBS AG" ,"UNITED OVERSEAS BANK LTD" ,"WOORI BANK", "SUMITOMO MITSUI BANKING CORPORATION", "WESTPAC BANKING CORPORATION", "PT BANK MAYBANK INDONESIA TBK", "DOHA BANK Q.P.S.C", "INDUSTRIAL BANK OF KOREA", "FIRST ABU DHABI BANK PJSC", "KEB HANA BANK", "Qatar National Bank (Q.P.S.C.)", "EMIRATES NBD BANK (P.J.S.C.)", "BANK OF CHINA LIMITED", "KOOKMIN BANK"]
Private_Banks = ["AXIS BANK LIMITED", "CITY UNION BANK LIMITED", "CSB BANK LIMITED", "DCB BANK LIMITED", "FEDERAL BANK LTD", "HDFC BANK LTD.", "ICICI BANK LIMITED", "IDBI BANK LIMITED", "INDUSIND BANK LTD", "ING VYSYA BANK LTD", "JAMMU & KASHMIR BANK LTD", "KARNATAKA BANK LTD" ,"KARUR VYSYA BANK LTD" ,"KOTAK MAHINDRA BANK LTD.", "LAKSHMI VILAS BANK LTD" ,"NAINITAL BANK LTD", "RBL BANK LIMITED", "SOUTH INDIAN BANK LTD", "TAMILNAD MERCANTILE BANK LTD", "THE DHANALAKSHMI BANK LTD" ,"YES BANK LTD.", "IDFC FIRST BANK LIMITED", "BANDHAN BANK LIMITED"]
Public_Banks = ["ALLAHABAD BANK", "ANDHRA BANK", "BANK OF BARODA", "BANK OF INDIA", "BANK OF MAHARASHTRA", "CANARA BANK", "CENTRAL BANK OF INDIA", "CORPORATION BANK", "DENA BANK", "IDBI BANK LIMITED", "INDIAN BANK", "INDIAN OVERSEAS BANK", "ORIENTAL BANK OF COMMERCE", "PUNJAB AND SIND BANK", "PUNJAB NATIONAL BANK", "STATE BANK OF BIKANER AND JAIPUR", "STATE BANK OF HYDERABAD", "STATE BANK OF INDIA", "STATE BANK OF MYSORE", "STATE BANK OF PATIALA", "STATE BANK OF TRAVANCORE", "SYNDICATE BANK", "UCO BANK", "UNION BANK OF INDIA", "UNITED BANK OF INDIA", "VIJAYA BANK", "BHARATIYA MAHILA BANK LTD."]
Small_Finance = ["CAPITAL SMALL FINANCE BANK LIMITED", "EQUITAS SMALL FINANCE BANK LIMITED", "ESAF SMALL FINANCE BANK LIMITED", "FINCARE SMALL FINANCE BANK LIMITED", "SURYODAY SMALL FINANCE BANK LIMITED", "UJJIVAN SMALL FINANCE BANK LIMITED", "AU SMALL FINANCE BANK LIMITED", "NORTH EAST SMALL FINANCE BANK LIMITED", "UTKARSH SMALL FINANCE BANK LIMITED", "JANA SMALL FINANCE BANK LIMITED"]
All_Banks = Foreign_Banks + Private_Banks + Public_Banks + Small_Finance
print(f"{len(Foreign_Banks)}, {len(Private_Banks)} , {len(Public_Banks)}, {len(Small_Finance)}, {len(All_Banks)}")

51, 23 , 27, 10, 111


In [5]:
def clean_dbie(df, x):
    if x==2: # 2 for how the excel sheet column names are formatted, using two rows for one column name
        df_1 = df.iloc[~df.index.isin([0, 1, len(df)-2, len(df) - 1])]
        df_1.pop('Unnamed: 0')
        row_index_to_transform = 0
        row_to_transform = df_1.iloc[row_index_to_transform]
        row_filled = row_to_transform.ffill()
        transformed_row = pd.DataFrame(row_filled).T
        df_2 = pd.concat([transformed_row, df_1.drop(2)], ignore_index=True)   
        new_columns = [f"{df_2.iloc[0, i]}_{df_2.iloc[1, i]}" for i in range(0,df_2.shape[1])]
        df_2.columns = new_columns
        df_3 = df_2.iloc[~df_2.index.isin([0, 1])]
        new_names = ['Quarter', 'Banks']
        df_3.columns = new_names + list(df_3.columns[2:])
        df_3.reset_index(inplace=True)
        df_3 = df_3.drop("index", axis = 1)
        df_3.at[df_3.shape[0] - 1, 'Quarter'] = np.nan
        column_to_transform = 'Quarter'
        col_to_transform = df_3[column_to_transform]
        col_to_transform_filled = col_to_transform.ffill()
        transformed_col = pd.DataFrame(col_to_transform_filled)
        df_4 = pd.concat([transformed_col, df_3.drop(columns=column_to_transform)], axis=1)
        df_4['Quarter'] = df_4['Quarter'].dt.to_period(freq='Q')
        values_to_remove = ['JSC VTB BANK', 'Commercial Banks', 'Foreign Bank Group', 'Private Sector Banks', 'Public Sector Banks', 'Small Finance Bank Group-Scheduled', 'All Scheduled Commercial Banks']
        mask = df_4.isin(values_to_remove).any(axis=1)
        df_5 = df_4[~mask]
        df_5['Foreign_Dummy'] = df_5['Banks'].isin(Foreign_Banks).astype(int)
        df_5['Private_Dummy'] = df_5['Banks'].isin(Private_Banks).astype(int)
        df_5['Public_Dummy'] = df_5['Banks'].isin(Public_Banks).astype(int)
        df_5['Small_Finance_Dummy'] = df_5['Banks'].isin(Small_Finance).astype(int)
        df_5.reset_index(inplace=True)
        df_5 = df_5.drop("index", axis = 1)
        return df_5
    elif x == 3: # 3 for how the excel sheet column names are formatted, using two rows for one column name
        df_1 = df.iloc[~df.index.isin([0, 1, 2, len(df)-2, len(df) - 1])]
        df_1.pop('Unnamed: 0')
        row_index_to_transform = 0
        row_to_transform = df_1.iloc[row_index_to_transform]
        row_filled = row_to_transform.ffill()
        transformed_row = pd.DataFrame(row_filled).T
        df_2 = pd.concat([transformed_row, df_1.drop(3)], ignore_index=True)   
        new_columns = [f"{df_2.iloc[0, i]}_{df_2.iloc[1, i]}" for i in range(0,df_2.shape[1])]
        df_2.columns = new_columns
        df_3 = df_2.iloc[~df_2.index.isin([0, 1])]
        new_names = ['Quarter', 'Banks']
        df_3.columns = new_names + list(df_3.columns[2:])
        df_3.reset_index(inplace=True)
        df_3 = df_3.drop("index", axis = 1)
        df_3.at[df_3.shape[0] - 1, 'Quarter'] = np.nan
        column_to_transform = 'Quarter'
        col_to_transform = df_3[column_to_transform]
        col_to_transform_filled = col_to_transform.ffill()
        transformed_col = pd.DataFrame(col_to_transform_filled)
        df_4 = pd.concat([transformed_col, df_3.drop(columns=column_to_transform)], axis=1)
        df_4['Quarter'] = df_4['Quarter'].dt.to_period(freq='Q')
        values_to_remove = ['JSC VTB BANK', 'Commercial Banks', 'Foreign Bank Group', 'Private Sector Banks', 'Public Sector Banks', 'Small Finance Bank Group-Scheduled', 'All Scheduled Commercial Banks']
        mask = df_4.isin(values_to_remove).any(axis=1)
        df_5 = df_4[~mask]
        df_5['Foreign_Dummy'] = df_5['Banks'].isin(Foreign_Banks).astype(int)
        df_5['Private_Dummy'] = df_5['Banks'].isin(Private_Banks).astype(int)
        df_5['Public_Dummy'] = df_5['Banks'].isin(Public_Banks).astype(int)
        df_5['Small_Finance_Dummy'] = df_5['Banks'].isin(Small_Finance).astype(int)
        df_5.reset_index(inplace=True)
        df_5 = df_5.drop("index", axis = 1)
        return df_5
    elif x == 'ms': # for money stock data
        df_1 = df.iloc[~df.index.isin([0, 1, 2, 3, len(df)-2, len(df) - 1])]
        df_1.pop('Unnamed: 0')
        row_index_to_transform = 0
        row_to_transform = df_1.iloc[row_index_to_transform]
        row_filled = row_to_transform.ffill()
        transformed_row = pd.DataFrame(row_filled).T
        df_2 = pd.concat([transformed_row, df_1.drop(4)], ignore_index=True)   
        new_columns = [f"{df_2.iloc[0, i]}_{df_2.iloc[1, i]}" for i in range(0,df_2.shape[1])]
        df_2.columns = new_columns
        df_3 = df_2.iloc[~df_2.index.isin([0, 1])]
        new_names = ['Date', 'Banks']
        df_3.columns = new_names + list(df_3.columns[2:])
        df_3.reset_index(inplace=True)
        df_3 = df_3.drop("index", axis = 1)
        return df_3
    else: 
        return print("Error wrong x")

In [6]:
balance_sheet = clean_dbie(all_sheets['Major Balance Sheet Items'], 2)
capital_adequacy = clean_dbie(all_sheets['Capital Adequacy'], 2)
asset_quality = clean_dbie(all_sheets['Assest Quality'], 2)
profitability = clean_dbie(all_sheets['Profitability'], 3)
performance_indicators = clean_dbie(all_sheets['Performance Indicators'], 2)
major_sectoral = clean_dbie(all_sheets['Major Sectoral'], 3)
priority_break_up = clean_dbie(all_sheets['Priority-Non_Priority Break-up'], 3)
infrastructure = clean_dbie(all_sheets['Infrastructure Financing Items'], 2)
money_stock = clean_dbie(ms, 'ms')

/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_37255/843860718.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5['Foreign_Dummy'] = df_5['Banks'].isin(Foreign_Banks).astype(int)
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_37255/843860718.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5['Private_Dummy'] = df_5['Banks'].isin(Private_Banks).astype(int)
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_37255/843860718.py:29: SettingWithCopyWarning: 
A value i

In [7]:
balance_sheet_2 = clean_dbie(all_sheets_2['Major Balance Sheet Items'], 2)
capital_adequacy_2 = clean_dbie(all_sheets_2['Capital Adequacy'], 2)
asset_quality_2 = clean_dbie(all_sheets_2['Assest Quality'], 2)
profitability_2 = clean_dbie(all_sheets_2['Profitability'], 3)
performance_indicators_2 = clean_dbie(all_sheets_2['Performance Indicators'], 2)
major_sectoral_2 = clean_dbie(all_sheets_2['Major Sectoral'], 3)
priority_break_up_2 = clean_dbie(all_sheets_2['Priority-Non_Priority Break-up'], 3)
infrastructure_2 = clean_dbie(all_sheets_2['Infrastructure Financing Items'], 2)

/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_37255/843860718.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5['Foreign_Dummy'] = df_5['Banks'].isin(Foreign_Banks).astype(int)
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_37255/843860718.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5['Private_Dummy'] = df_5['Banks'].isin(Private_Banks).astype(int)
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_37255/843860718.py:29: SettingWithCopyWarning: 
A value i

In [8]:
balance_sheet_cov = pd.concat([balance_sheet, balance_sheet_2])
capital_adequacy_cov = pd.concat([capital_adequacy, capital_adequacy_2])
asset_quality_cov = pd.concat([asset_quality, asset_quality_2])
profitability_cov = pd.concat([profitability, profitability_2])
performance_indicators_cov = pd.concat([performance_indicators, performance_indicators_2])
major_sectoral_cov = pd.concat([major_sectoral, major_sectoral_2])
priority_break_up_cov = pd.concat([priority_break_up, priority_break_up_2])
infrastructure_cov = pd.concat([infrastructure, infrastructure_2])

In [9]:
cred_deploy
df_1 = cred_deploy.iloc[~cred_deploy.index.isin([0, 1, 2, 3, 4, 49, 50, 51, 52])]
df_1.pop('Unnamed: 0')
new_columns = [f"{df_1.iloc[0, i]}" for i in range(0,df_1.shape[1])]
df_1.columns = new_columns
col = df_1.columns
col_2 = col[2:]
col_3 = list(col_2)
col_3.insert(0, 'Serial No')
col_3.insert(1, 'Industry')
col_3
df_1.columns = col_3
df_2 = df_1.iloc[~df_1.index.isin([5])]
df_2.at[47, 'Serial No'] = '2.18.7'
cred_deploy = df_2

In [10]:
df_3 = cred_deploy.T
new_columns = [f"{df_3.iloc[1,i]}({df_3.iloc[0, i]})" for i in range(0,df_3.shape[1])]
df_3.columns = new_columns
df_3 = df_3.iloc[~df_3.index.isin(['Serial No', 'Industry'])]
df_3 = df_3.reset_index()
cred_deploy = df_3

In [11]:
cred_deploy

,index,Industries (2.1 to 2.19)(2 ),Mining & Quarrying (incl. Coal)(2.1 ),Food Processing(2.2 ),Sugar(2.2.1 ),Edible Oils & Vanaspati(2.2.2 ),Tea(2.2.3 ),Others(2.2.4 ),Beverage & Tobacco(2.3 ),Textiles(2.4 ),...,Construction(2.17 ),Infrastructure(2.18 ),Power(2.18.1 ),Telecommunications(2.18.2 ),Roads(2.18.3 ),Airports(2.18.4 ),Ports(2.18.5 ),Railways(2.18.6 ),Other Infrastructure(2.18.7),Other Industries(2.19 )
0,"June, 29 2012",NaN,334850000000,927360000000,297930000000,124530000000,22160000000,482740000000,146490000000,1613790000000,...,497510000000,6484470000000,3419000000000,976180000000,1158570000000,NaN,NaN,NaN,930720000000,1980710000000
1,"September, 21 2012",NaN,322150000000,959480000000,273510000000,130870000000,26530000000,528580000000,152220000000,1633570000000,...,507270000000,6585030000000,3644760000000,868840000000,1228790000000,NaN,NaN,NaN,842640000000,1859300000000
2,"December, 28 2012",NaN,350230000000,1046100000000,265150000000,159660000000,25870000000,595420000000,159550000000,1706810000000,...,490660000000,6980560000000,3864600000000,931200000000,1265110000000,NaN,NaN,NaN,919640000000,1850020000000
3,"March, 22 2013",NaN,346390000000,1173680000000,329760000000,170530000000,25820000000,647570000000,165110000000,1835360000000,...,521660000000,7297210000000,4158490000000,877650000000,1313120000000,NaN,NaN,NaN,947960000000,1809680000000
4,"June, 28 2013",NaN,317220700000,1231164000000,337914100000,182700700000,27309700000,683239500000,156147600000,1837244200000,...,560356300000,7695997100000,4427325800000,889583700000,1411222200000,NaN,NaN,NaN,967865400000,1770645600000
5,"September, 20 2013",NaN,328890000000,1201530000000,317830000000,176150000000,33290000000,674270000000,159000000000,1855500000000,...,565260000000,7931970000000,4591210000000,873620000000,1437020000000,NaN,NaN,NaN,1030120000000,1841650000000
6,"December, 27 2013",NaN,352150000000,1350330000000,296960000000,219200000000,32550000000,801630000000,163640000000,1927910000000,...,577240000000,8037150000000,4683530000000,856260000000,1510660000000,NaN,NaN,NaN,986690000000,1802620000000
7,"March, 21 2014",25164840000000,358450000000,1462540000000,344660000000,212930000000,32410000000,872530000000,182860000000,2022130000000,...,625710000000,8363560000000,4869020000000,882040000000,1578600000000,NaN,NaN,NaN,1033900000000,1880600000000
8,"June, 27 2014",NaN,373540000000,1487320000000,356340000000,207250000000,29000000000,894730000000,182590000000,1979370000000,...,705020000000,8553630000000,5065570000000,827950000000,1619460000000,NaN,NaN,NaN,1040650000000,1784970000000
9,"September, 19 2014",NaN,385080000000,1408610000000,338700000000,176570000000,31060000000,862270000000,192110000000,1933670000000,...,724540000000,8713090000000,5178410000000,851460000000,1639630000000,NaN,NaN,NaN,1043580000000,1752980000000


In [12]:
cred_deploy_2
df_1 = cred_deploy_2.iloc[~cred_deploy_2.index.isin([0,1,2,3,4,50,51,52,53])]
df_1.pop('Unnamed: 0')
new_columns = [f"{df_1.iloc[0, i]}" for i in range(0,df_1.shape[1])]
df_1.columns = new_columns
col = df_1.columns
col_2 = col[2:]
col_3 = list(col_2)
col_3.insert(0, 'Serial No')
col_3.insert(1, 'Industry')
col_3
df_1.columns = col_3
df_2 = df_1.iloc[~df_1.index.isin([5])]
cred_deploy_2 = df_2
#cred_deploy_2

In [13]:
df_3 = cred_deploy_2.T
new_columns = [f"{df_3.iloc[1,i]}({df_3.iloc[0, i]} )" for i in range(0,df_3.shape[1])]
df_3.columns = new_columns
df_3 = df_3.iloc[~df_3.index.isin(['Serial No', 'Industry'])]
df_3 = df_3.reset_index()
cred_deploy_2 = df_3
cred_deploy_2.pop('nan(nan )')
cols = cred_deploy.columns
cred_deploy_2.columns = cols
cred_deploy_2

,index,Industries (2.1 to 2.19)(2 ),Mining & Quarrying (incl. Coal)(2.1 ),Food Processing(2.2 ),Sugar(2.2.1 ),Edible Oils & Vanaspati(2.2.2 ),Tea(2.2.3 ),Others(2.2.4 ),Beverage & Tobacco(2.3 ),Textiles(2.4 ),...,Construction(2.17 ),Infrastructure(2.18 ),Power(2.18.1 ),Telecommunications(2.18.2 ),Roads(2.18.3 ),Airports(2.18.4 ),Ports(2.18.5 ),Railways(2.18.6 ),Other Infrastructure(2.18.7),Other Industries(2.19 )
0,"March, 27 2020",2946851.25,45406.79,144268.84,27563.45,16521.68,4929.18,95254.53,18025.49,193452.92,...,127837.59,1083656.43,577326.55,147176.4,181530.8,5143.87,12745.18,10884.73,148848.9,223620.5
1,"June, 19 2020",2911362.69,42253.49,149015.3,26488.3,15629.29,4634.19,102263.52,16533.28,190784.86,...,127892.76,1101626.86,586633.82,150688.11,182055.92,4802.74,14078.11,12174.22,151193.94,205511.88
2,"September, 25 2020",2831679.89,40193.13,139617.66,21317.08,16223.47,5123.63,96953.48,16399.04,189635.98,...,128783.5,1059541.12,564767.64,122346.36,198406.07,5453.63,11755.45,13278.85,143533.12,201124.66
3,"December, 18 2020",2822924.36,42295.59,157785.66,27166.33,17239.32,5278.93,108101.08,15952.44,193322.58,...,127239.73,1043335.55,564109.36,104087.63,202368.61,6478.51,10927.88,12370.93,142992.63,201002.13
4,"March, 26 2021",2934688.691812,43498.190103,156556.826727,26490.400381,17998.84563,5059.66,107007.920716,17804.609372,204971.775735,...,119670.341604,1095467.252598,571028.299087,114960.820779,226299.436161,8573.41,10152.8,12471.11,151981.37657,229431.722417
5,"June, 18 2021",2963016.767421,46552.8493,157335.409076,24002.679447,17437.109093,5389.78,110505.840536,17381.309306,213947.095669,...,124144.353364,1115392.355526,574741.017201,117646.35,238663.130724,9724.12,10389.69,12610.97,151617.077601,206514.197401
6,"September, 24 2021",2953548.022651,48985.889006,151709.299469,19156.628098,16977.779758,5710.56,109864.331613,16837.415474,210523.275347,...,122209.534756,1105033.908499,578801.107737,113414.806633,248586.773064,7593.54,11371.69,13004.32,132261.671065,219683.873383
7,"December, 31 2021",3106358.322852,50962.555427,170429.738847,21427.81473,18063.6129,6077.71,124860.601217,17655.44622,224449.750184,...,120142.261055,1148724.801528,600476.84858,109350.417652,255328.148632,7624.29,9453.43,10193.89,156297.776664,249794.36022
8,"March, 25 2022",3235278.907314,51129.797441,176658.02019,26370.365018,18434.094944,5818.129499,126035.43073,18674.63327,231700.755866,...,122454.776842,1205737.164518,615806.43019,130614.777591,273197.384462,6707.98,8897.95,10604.97,159907.672275,256126.893877
9,"June, 17 2022",3248920.985505,50514.627224,179398.103837,24157.393644,18559.856727,6459.828708,130221.024758,17912.841907,226293.301606,...,123010.935846,1222190.250635,624747.780735,132251.103169,281206.062002,6870.16,7964.87,11588.34,157561.934729,251550.947308


In [14]:
cred_deploy_cov = pd.concat([cred_deploy, cred_deploy_2])

In [15]:
# # Uncomment to write files to csv
# balance_sheet.to_csv('Clean Data/balance_sheet.csv')
# capital_adequacy.to_csv('Clean Data/capital_adequacy.csv')
# asset_quality.to_csv('Clean Data/asset_quality.csv')
# profitability.to_csv('Clean Data/profitability.csv')
# performance_indicators.to_csv('Clean Data/performance_indicators.csv')
# major_sectoral.to_csv('Clean Data/major_sectoral.csv')
# priority_break_up.to_csv('Clean Data/priority_break_up.csv')
# infrastructure.to_csv('Clean Data/infrastructure.csv')
# money_stock.to_csv('Clean Data/money_stock.csv')
# cred_deploy.to_csv('Clean Data/cred_deploy.csv')

In [ ]:
# # Uncomment to write files to csv
# balance_sheet_cov.to_csv('Clean Data/balance_sheet_cov.csv')
# capital_adequacy_cov.to_csv('Clean Data/capital_adequacy_cov.csv')
# asset_quality_cov.to_csv('Clean Data/asset_quality_cov.csv')
# profitability_cov.to_csv('Clean Data/profitability_cov.csv')
# performance_indicators_cov.to_csv('Clean Data/performance_indicators_cov.csv')
# major_sectoral_cov.to_csv('Clean Data/major_sectoral_cov.csv')
# priority_break_up_cov.to_csv('Clean Data/priority_break_up_cov.csv')
# infrastructure_cov.to_csv('Clean Data/infrastructure_cov.csv')
# cred_deploy_cov.to_csv('Clean Data/cred_deploy_cov.csv')

In [18]:
with open('Raw Data/banknames.json', 'r') as file_2:
    data_2 = json.load(file_2)

In [19]:
bank_branch_code = pd.DataFrame([data_2])
bank_branch_code = bank_branch_code.T
bank_branch_code = bank_branch_code.reset_index()
bank_branch_code = bank_branch_code.rename(columns = { 'index' : 'Code', 0 : 'bank_name'})
branch_name_list = bank_branch_code['bank_name'].to_list()
branch_name_list_cap = [x.upper() for x in branch_name_list]
branch_name_list_cap

['ABHYUDAYA COOPERATIVE BANK',
 'ABU DHABI COMMERCIAL BANK',
 'AHMEDABAD MERCANTILE COOPERATIVE BANK',
 'AKOLA JANATA COMMERCIAL COOPERATIVE BANK',
 'ALLAHABAD BANK',
 'ALMORA URBAN CO-OPERATIVE BANK LTD',
 'ANDHRA BANK',
 'ANDHRA PRAGATHI GRAMEENA BANK',
 'APNA SAHAKARI BANK LTD',
 'AUSTRALIA AND NEW ZEALAND BANKING GROUP LTD',
 'AXIS BANK',
 'BANDHAN BANK LIMITED',
 'PT BANK MAYBANK INDONESIA TBK',
 'BANK OF AMERICA',
 'BANK OF BAHREIN AND KUWAIT',
 'BANK OF BARODA',
 'BANK OF CEYLON',
 'BANK OF INDIA',
 'BANK OF MAHARASHTRA',
 'BANK OF TOKYO MITSUBISHI LTD',
 'BARCLAYS BANK',
 'BASSEIN CATHOLIC CO-OP BANK LTD',
 'BHARAT COOPERATIVE BANK MUMBAI LTD',
 'BHARATIYA MAHILA BANK LTD',
 'BNP PARIBAS BANK',
 'CANARA BANK',
 'CAPITAL LOCAL AREA BANK LTD',
 'CATHOLIC SYRIAN BANK',
 'CENTRAL BANK OF INDIA',
 'CHINATRUST COMMERCIAL BANK',
 'CITI BANK',
 'CITIZEN CREDIT COOPERATIVE BANK',
 'CITY UNION BANK LTD',
 'COMMONWEALTH BANK OF AUSTRALIA',
 'CORPORATION BANK',
 'CREDIT AGRICOLE CORPORATE 

In [20]:
bank_branch_code

,Code,bank_name
0,ABHY,Abhyudaya Cooperative Bank
1,ADCB,Abu Dhabi Commercial Bank
2,AMCB,Ahmedabad Mercantile Cooperative Bank
3,AKJB,Akola Janata Commercial Cooperative Bank
4,ALLA,Allahabad Bank
...,...,...
173,VIJB,Vijaya Bank
174,WPAC,Westpac Banking Corporation
175,HVBK,Woori Bank
176,YESB,Yes Bank Ltd


In [21]:
Foreign_Banks_2 = ["AB BANK LTD", "ABU DHABI COMMERCIAL BANK" ,"AMERICAN EXPRESS BANKING CORP", "AUSTRALIA AND NEW ZEALAND BANKING GROUP LTD" ,"BANK OF AMERICA" ,"BANK OF BAHREIN AND KUWAIT" ,"BANK OF CEYLON", "BANK OF NOVA SCOTIA", "BARCLAYS BANK" ,"BNP PARIBAS BANK" ,"CITI BANK" ,"COMMONWEALTH BANK OF AUSTRALIA", "COOPERATIEVE RABOBANK U.A." ,"CREDIT AGRICOLE CORPORATE AND INVESTMENT BANK", "CREDIT SUISSE AG", "CTBC BANK CO., LTD" ,"DBS BANK INDIA LTD", "DEUTSCHE BANK" ,"FIRSTRAND BANK LTD" ,"HONGKONG AND SHANGHAI BANKING CORPN.LTD" ,"HSBC BANK OMAN S.A.O.G." ,"INDUSTRIAL AND COMMERCIAL BANK OF CHINA LTD", "JP MORGAN CHASE BANK NA", "JSC VTB BANK", "KBC BANK NV" ,"KRUNG THAI BANK PUBLIC COMPANY LIMITED" ,"MASHREQ BANK", "MIZUHO CORPORATE BANK LTD", "MUFG BANK LTD", "NATIONAL AUSTRALIA BANK", "NatWest Markets Plc", "SBER BANK" ,"SBM BANK (INDIA) LTD" ,"SHINHAN BANK" ,"SOCIETE GENERALE" ,"SONALI BANK", "STANDARD CHARTERED BANK" ,"UBS AG" ,"UNITED OVERSEAS BANK" ,"WOORI BANK", "SUMITOMO MITSUI BANKING CORPORATION", "WESTPAC BANKING CORPORATION", "PT BANK MAYBANK INDONESIA TBK", "DOHA BANK QSC", "INDUSTRIAL BANK OF KOREA", "FIRST ABU DHABI BANK PJSC", "KEB HANA BANK", "Qatar National Bank (Q.P.S.C.)", "EMIRATES NBD BANK (P.J.S.C.)", "BANK OF CHINA LTD", "KOOKMIN BANK"]
Private_Banks_2 = ["AXIS BANK", "CITY UNION BANK LTD", "CATHOLIC SYRIAN BANK", "DCB BANK LTD", "FEDERAL BANK LTD", "HDFC BANK LTD", "ICICI BANK LTD", "IDBI LTD", "INDUSIND BANK LTD", "ING VYSYA BANK LTD", "JAMMU & KASHMIR BANK", "KARNATAKA BANK LTD" ,"KARUR VYSYA BANK" ,"KOTAK MAHINDRA BANK", "LAXMI VILAS BANK" ,"NAINITAL BANK LTD", "RBL BANK LIMITED", "SOUTH INDIAN BANK", "TAMILNADU MERCANTILE BANK", "DHANLAXMI BANK LTD" ,"YES BANK LTD", "IDFC BANK LTD", "BANDHAN BANK LIMITED"]
Public_Banks_2 = ["ALLAHABAD BANK", "ANDHRA BANK", "BANK OF BARODA", "BANK OF INDIA", "BANK OF MAHARASHTRA", "CANARA BANK", "CENTRAL BANK OF INDIA", "CORPORATION BANK", "DENA BANK", "IDBI LTD", "INDIAN BANK", "INDIAN OVERSEAS BANK", "ORIENTAL BANK OF COMMERCE", "PUNJAB AND SIND BANK", "PUNJAB NATIONAL BANK", "STATE BANK OF BIKANER AND JAIPUR", "STATE BANK OF HYDERABAD", "STATE BANK OF INDIA", "STATE BANK OF MYSORE", "STATE BANK OF PATIALA", "STATE BANK OF TRAVANCORE", "SYNDICATE BANK", "UCO BANK", "UNION BANK OF INDIA", "UNITED BANK OF INDIA", "VIJAYA BANK", "BHARATIYA MAHILA BANK LTD"]
Small_Finance_2= ["CAPITAL SMALL FINANCE BANK LTD", "EQUITAS SMALL FINANCE BANK LIMITED", "ESAF SMALL FINANCE BANK LTD", "FINCARE SMALL FINANCE BANK LTD", "SURYODAY SMALL FINANCE BANK LTD", "UJJIVAN SMALL FINANCE BANK LTD", "AU SMALL FINANCE BANK LTD", "NORTH EAST SMALL FINANCE BANK LTD", "UTKARSH SMALL FINANCE BANK LTD", "JANA SMALL FINANCE BANK LTD"]
All_Banks_2 = Foreign_Banks_2 + Private_Banks_2 + Public_Banks_2 + Small_Finance_2
All_Banks_2 = [x.upper() for x in All_Banks_2]
All_Banks_2

['AB BANK LTD',
 'ABU DHABI COMMERCIAL BANK',
 'AMERICAN EXPRESS BANKING CORP',
 'AUSTRALIA AND NEW ZEALAND BANKING GROUP LTD',
 'BANK OF AMERICA',
 'BANK OF BAHREIN AND KUWAIT',
 'BANK OF CEYLON',
 'BANK OF NOVA SCOTIA',
 'BARCLAYS BANK',
 'BNP PARIBAS BANK',
 'CITI BANK',
 'COMMONWEALTH BANK OF AUSTRALIA',
 'COOPERATIEVE RABOBANK U.A.',
 'CREDIT AGRICOLE CORPORATE AND INVESTMENT BANK',
 'CREDIT SUISSE AG',
 'CTBC BANK CO., LTD',
 'DBS BANK INDIA LTD',
 'DEUTSCHE BANK',
 'FIRSTRAND BANK LTD',
 'HONGKONG AND SHANGHAI BANKING CORPN.LTD',
 'HSBC BANK OMAN S.A.O.G.',
 'INDUSTRIAL AND COMMERCIAL BANK OF CHINA LTD',
 'JP MORGAN CHASE BANK NA',
 'JSC VTB BANK',
 'KBC BANK NV',
 'KRUNG THAI BANK PUBLIC COMPANY LIMITED',
 'MASHREQ BANK',
 'MIZUHO CORPORATE BANK LTD',
 'MUFG BANK LTD',
 'NATIONAL AUSTRALIA BANK',
 'NATWEST MARKETS PLC',
 'SBER BANK',
 'SBM BANK (INDIA) LTD',
 'SHINHAN BANK',
 'SOCIETE GENERALE',
 'SONALI BANK',
 'STANDARD CHARTERED BANK',
 'UBS AG',
 'UNITED OVERSEAS BANK',
 'W

In [22]:
inter_1 = []
All_Banks_3 = [x.upper() for x in All_Banks]
for i in All_Banks_3:
    if i in branch_name_list_cap:
        inter_1.append(i)

len(inter_1)

46

In [23]:
inter_2 = []
for i in All_Banks_2:
    if i not in branch_name_list_cap:
        inter_2.append(i)
inter_2

['AB BANK LTD',
 'AMERICAN EXPRESS BANKING CORP',
 'BANK OF NOVA SCOTIA',
 'COOPERATIEVE RABOBANK U.A.',
 'CTBC BANK CO., LTD',
 'DBS BANK INDIA LTD',
 'HONGKONG AND SHANGHAI BANKING CORPN.LTD',
 'HSBC BANK OMAN S.A.O.G.',
 'JSC VTB BANK',
 'KBC BANK NV',
 'KRUNG THAI BANK PUBLIC COMPANY LIMITED',
 'MUFG BANK LTD',
 'NATWEST MARKETS PLC',
 'SBM BANK (INDIA) LTD',
 'SONALI BANK',
 'UBS AG',
 'FIRST ABU DHABI BANK PJSC',
 'QATAR NATIONAL BANK (Q.P.S.C.)',
 'EMIRATES NBD BANK (P.J.S.C.)',
 'BANK OF CHINA LTD',
 'KOOKMIN BANK',
 'ING VYSYA BANK LTD',
 'JAMMU & KASHMIR BANK',
 'NAINITAL BANK LTD',
 'CAPITAL SMALL FINANCE BANK LTD',
 'ESAF SMALL FINANCE BANK LTD',
 'FINCARE SMALL FINANCE BANK LTD',
 'SURYODAY SMALL FINANCE BANK LTD',
 'UJJIVAN SMALL FINANCE BANK LTD',
 'AU SMALL FINANCE BANK LTD',
 'NORTH EAST SMALL FINANCE BANK LTD',
 'UTKARSH SMALL FINANCE BANK LTD',
 'JANA SMALL FINANCE BANK LTD']

In [24]:
inter_3 = []
for i in All_Banks_2:
    if i in branch_name_list_cap:
        inter_3.append(i)

len(inter_3)

78

In [25]:
inter_4 = []
for i in All_Banks_2:
    if i not in branch_name_list_cap:
        inter_4.append(i)
len(inter_4)

33

In [26]:
inter_5 = []
for i in branch_name_list_cap:
    if i in All_Banks_2:
        inter_5.append(i)

len(inter_5)

77

In [27]:
bank_branch_code['bank_name'] = bank_branch_code['bank_name'].str.upper()
bank_branch_code

,Code,bank_name
0,ABHY,ABHYUDAYA COOPERATIVE BANK
1,ADCB,ABU DHABI COMMERCIAL BANK
2,AMCB,AHMEDABAD MERCANTILE COOPERATIVE BANK
3,AKJB,AKOLA JANATA COMMERCIAL COOPERATIVE BANK
4,ALLA,ALLAHABAD BANK
...,...,...
173,VIJB,VIJAYA BANK
174,WPAC,WESTPAC BANKING CORPORATION
175,HVBK,WOORI BANK
176,YESB,YES BANK LTD


In [28]:
bank_branch_code_filter = bank_branch_code[bank_branch_code['bank_name'].isin(inter_3)]
bank_branch_code_filter

,Code,bank_name
1,ADCB,ABU DHABI COMMERCIAL BANK
4,ALLA,ALLAHABAD BANK
6,ANDB,ANDHRA BANK
9,ANZB,AUSTRALIA AND NEW ZEALAND BANKING GROUP LTD
10,UTIB,AXIS BANK
...,...,...
171,UOVB,UNITED OVERSEAS BANK
173,VIJB,VIJAYA BANK
174,WPAC,WESTPAC BANKING CORPORATION
175,HVBK,WOORI BANK


In [29]:
bank_branch_code_filter = bank_branch_code_filter[bank_branch_code_filter['Code'] != 'ESFB'] #Not in branch locations

In [30]:
banks_to_read = bank_branch_code_filter['Code']
banks_to_read

1      ADCB
4      ALLA
6      ANDB
9      ANZB
10     UTIB
       ... 
171    UOVB
173    VIJB
174    WPAC
175    HVBK
176    YESB
Name: Code, Length: 76, dtype: object

In [31]:
data_dict = {}
for i in banks_to_read:
    with open(f'Raw Data/by-bank/{i}.json', 'r') as file:
        data_dict[i] = json.load(file)

In [32]:
len(data_dict)

76

In [33]:
for i in banks_to_read:
    globals()[f'bank_branch_{i}'] = pd.json_normalize(data_dict[i].values())

In [34]:
bank_branch_HDFC

,BANK,IFSC,BRANCH,ADDRESS,CONTACT,CITY,DISTRICT,STATE
0,HDFC BANK,HDFC0000001,TULSIANI CHMBRS - NARIMAN PT,101-104 TULSIANI CHAMBERSFREE PRESS JOURNAL MA...,9890603333.0,MUMBAI,GREATER MUMBAI,MAHARASHTRA
1,HDFC BANK,HDFC0000002,MUMBAI - KHAR (WEST),"SWAGATAM,OPP. KHAR POLICE STATION,S.V.ROAD, KH...",9890603333.0,MUMBAI,GREATER MUMBAI,MAHARASHTRA
2,HDFC BANK,HDFC0000003,K G MARG,"209 - 214, KAILASH BUILDING, 26, KASTURBA GAND...",61606161,DELHI,NEW DELHI,DELHI
3,HDFC BANK,HDFC0000004,CHENNAI - ITC CENTRE - ANNA SALAI,"759, ITC CENTREANNA SALAI, OPP T.V.S.CHENNAITA...",9840673333.0,CHENNAI,CHENNAI,TAMIL NADU
4,HDFC BANK,HDFC0000005,MUMBAI - BHULABHAI DESAI ROAD,"TIRUPATI APARTMENTS,SHOP NO. 4,BHULABHAI DESAI...",9890603333.0,MUMBAI,GREATER MUMBAI,MAHARASHTRA
...,...,...,...,...,...,...,...,...
5476,HDFC BANK,HDFC0CYACBP,THE YASHWANT CO-OP BANK LTD PHALTAN,"69,GURUWAR PETH,CHAWDI CHOWK,KARAD 415110",9422745818.0,KARAD,SATARA,MAHARASHTRA
5477,HDFC BANK,HDFC0CYDCBL,YOUTH DEVELOPMENT COOP BANK LTD.KOL,"POORNIMA F-2, FIRST FLOOR, 1429/G, NEAR SAMBHA...",2550120.0,KOLHAPUR,KOLHAPUR,MAHARASHTRA
5478,HDFC BANK,HDFC0CYMCBK,YEOLA MER CO O PBANK LTD,YASHODHAN BALAJI GALLI YEOLA DIST NASIK,265547.0,YEOLA,NASIK,MAHARASHTRA
5479,HDFC BANK,HDFC0CYMSB1,THE YAVATMAL MAHILA SAH BANK LTD,"ANNAPURNA, MAINDE CHOWK, SANKAT MOCHAN ROAD YA...",250661.0,YAVATMAL,YAVATMAL,MAHARASHTRA


In [35]:
branch_all = pd.concat([globals()[f'bank_branch_{i}'] for i in banks_to_read], ignore_index=True)
branch_all['foreign_dummy'] = branch_all['BANK'].isin(Foreign_Banks_2).astype(int)
branch_all['private_dummy'] = branch_all['BANK'].isin(Private_Banks_2).astype(int)
branch_all['public_dummy'] = branch_all['BANK'].isin(Public_Banks_2).astype(int)
branch_all['small_finance_dummy'] = branch_all['BANK'].isin(Small_Finance_2).astype(int)

In [36]:
branch_all

,BANK,IFSC,BRANCH,ADDRESS,CONTACT,CITY,DISTRICT,STATE,foreign_dummy,private_dummy,public_dummy,small_finance_dummy
0,ABU DHABI COMMERCIAL BANK,ADCB0000001,RTGS-HO,"75, REHMAT MANZIL, V. N. ROAD, CURCHGATE, MUMB...",39534100,MUMBAI,MUMBAI CITY,MAHARASHTRA,1,0,0,0
1,ABU DHABI COMMERCIAL BANK,ADCB0000002,BANGALORE,"CITI CENTRE, 28, CHURCH STREET, OFF M. G. ROAD...",25582000,BANGALORE,BANGALORE URBAN,KARNATAKA,1,0,0,0
2,ALLAHABAD BANK,ALLA0210001,GUWAHATI MAIN BRANCH,"H. B. ROAD, MACHOWA, OPP. OF ASTC BUS STAND MA...",2545218,GUWAHATI,KAMRUP,ASSAM,0,0,1,0
3,ALLAHABAD BANK,ALLA0210002,PATNA UNIVERSITY,"PATNA UNIVERSITY.GANESH ROAD,DIST-PATNA",2671644,PATNA,PATNA,BIHAR,0,0,1,0
4,ALLAHABAD BANK,ALLA0210003,PATNA MAIN,"BUDH MARG,DIST-PATNA,PIN-800001",2235237,PATNA,PATNA,BIHAR,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
121035,YES BANK,YESB0ZSBL20,ZILA SAHKARI BANK LUCKNOW ICCMRT EC,"21/467, INDIRA NAGAR, LUCKNOW - 226016",8756400236.0,LUCKNOW,LUCKNOW,UTTAR PRADESH,0,0,0,0
121036,YES BANK,YESB0ZSBL21,ZILA SAHKARI BANK LUCKNOW PARAGDARY,"JOPPLING ROAD, LUCKNOW - 226001",8756400237.0,LUCKNOW,LUCKNOW,UTTAR PRADESH,0,0,0,0
121037,YES BANK,YESB0ZSBL22,ZILA SAHKARI BANK LUCKNOW ARJUNGANJ,"AHMAMAU, ARJUNGANJ, LUCKNOW - 226002",8756400239.0,ARJUNGANJ,LUCKNOW,UTTAR PRADESH,0,0,0,0
121038,YES BANK,YESB0ZSBL23,ZILA SAHKARI BANK LUCKNOW CHANDRNGR,"H NO 551 KA/180 KHA,ST BANK RD, BHILLAWAN,CHAN...",8756400238.0,LUCKNOW,LUCKNOW,UTTAR PRADESH,0,0,0,0


In [ ]:
# #Uncomment to write to csv
# branch_all.to_csv(f'Clean Data/branch_all.csv')

In [38]:
with open('Raw Data/by-bank_cov/banknames.json', 'r') as file_3:
    data_3 = json.load(file_3)

In [39]:
bank_branch_code2 = pd.DataFrame([data_3])
bank_branch_code2 = bank_branch_code2.T
bank_branch_code2 = bank_branch_code2.reset_index()
bank_branch_code2 = bank_branch_code2.rename(columns = { 'index' : 'Code', 0 : 'bank_name'})
branch_name_list2 = bank_branch_code2['bank_name'].to_list()
branch_name_list_cap2 = [x.upper() for x in branch_name_list2]
branch_name_list_cap2 = sorted(branch_name_list_cap2)
branch_name_list_cap2

['510 ARMY BASE W/S CREDIT CO-OPERATIVE PRIMARY BANK',
 'A.P. MAHESH CO-OPERATIVE URBAN BANK',
 'AB BANK',
 'ABHINANDAN URBAN CO-OPERATIVE BANK AMRAVATI',
 'ABHINAV SAHAKARI BANK',
 'ABHYUDAYA CO-OPERATIVE BANK',
 'ABU DHABI COMMERCIAL BANK',
 'ACE CO-OPERATIVE BANK',
 'ADARNIYA P.D. PATILSAHEB SAHAKARI BANK',
 'ADARSH CO-OPERATIVE BANK',
 'ADARSH CO-OPERATIVE URBAN BANK',
 'ADARSH MAHILA MERCANTILE CO-OPERATIVE BANK',
 'ADILABAD DISTRICT CO-OPERATIVE CENTRAL BANK',
 'ADINATH CO-OPERATIVE BANK',
 'ADITYA BIRLA IDEA PAYMENTS BANK',
 'ADV.SHAMRAOJI SHINDE SATYASHODHAK BANK',
 'AGARTALA CO-OPERATIVE URBAN BANK',
 'AGRA DISTRICT CO-OPERATIVE BANK',
 'AGRASEN CO-OPERATIVE URBAN BANK',
 'AGRASEN NAGARI SAHAKARI BANK',
 'AGROHA CO-OPERATIVE URBAN BANK',
 'AHILYADEVI URBAN CO-OPERATIVE BANK LIMITED SOLAPUR',
 'AHMEDABAD DISTRICT CO-OPERATIVE BANK',
 'AHMEDABAD MERCANTILE CO-OPERATIVE BANK',
 'AHMEDNAGAR DISTRICT CENTRAL CO-OPERATIVE BANK',
 "AHMEDNAGAR MERCHANT'S CO-OPERATIVE BANK",
 'AHMEDNAG

In [40]:
branch_name_list_cap2_1 = branch_name_list_cap2[-500:]
branch_name_list_cap2_1

['PARBHANI DISTRICT CENTRAL CO-OPERATIVE BANK',
 'PARSHWANATH CO-OPERATIVE BANK',
 'PARWANOO URBAN CO-OPERATIVE BANK',
 'PASCHIM BANGA GRAMIN BANK',
 'PATAN CO-OPERATIVEEARTIVE BANK',
 'PATAN NAGARIK SAHAKARI BANK',
 'PATAN URBAN CO-OPERATIVE BANK PATAN',
 'PATHANMTHITTA DISTRICT CO-OPERATIVE BANK',
 'PATIALA CENTRAL CO-OPERATIVE BANK',
 'PATLIPUTRA CENTRAL CO-OPERATIVE BANK',
 'PAVANA SAHAKARI BANK',
 'PAYANGADI URBAN CO-OPERATIVE BANK',
 'PAYTM PAYMENTS BANK',
 'PAYYOLI CO-OPERATIVE URBAN BANK',
 'PEOPLES URBAN CO-OPERATIVE BANK',
 "PEOPLES' CO-OPERATIVE BANK",
 'PIMPALGAON MERCHANTS CO-OPERATIVE BANK',
 'PIMPRI CHINCHWAD SAHAKARI BANK MARYADIT,PIMPRI',
 'PITHORAGARH ZILA SAHKARI BANK',
 'POCHAMPALLY CO-OPERATIVE URBAN BANK',
 'PONDICHERRY STATE CO-OPERATIVE BANK',
 'POORNAWADI NAGRIK SAHAKARI BANK',
 'PRAGATHI CO-OPERATIVE BANK, BANGALORE',
 'PRAGATHI KRISHNA GRAMIN BANK',
 'PRAGATI CO-OPERATIVE BANK,THARA',
 'PRAGATI MAHILA NAGRIK SAHAKARI BANK BHILAI',
 'PRAGATI SAHAKARI BANK',
 '

In [41]:
len(branch_name_list_cap2)

1431

In [42]:
bank_branch_code2

,Code,bank_name
0,AACX,Akhand Anand Co.op Bank
1,ABBL,AB Bank
2,ABCX,Aurangabad District Central Co-operative Bank
3,ABDX,Dr. Ambedkar Nagrik Sahakari Bank Mydt Gwalior
4,ABEX,Andhra Bank Employees Co-operative Bank
...,...,...
1426,ZSHX,Zila Sahkari Bank Haridwar
1427,ZSJX,Zila Sahkari Bank Jhansi
1428,ZSKX,Zila Sahkari Bank Kanpur
1429,ZSLX,Zila Sahkari Bank Lakhimpur Kheri


In [43]:
inter_11 = []
for i in All_Banks_3:
    if i in branch_name_list_cap2:
        inter_11.append(i)

len(inter_11)

41

In [44]:
inter_21 = []
for i in All_Banks_2:
    if i in branch_name_list_cap2:
        inter_21.append(i)
len(inter_21)

59

In [45]:
Foreign_Banks_3 = ["AB BANK", "ABU DHABI COMMERCIAL BANK" ,"AMERICAN EXPRESS BANKING CORP", "AUSTRALIA AND NEW ZEALAND BANKING GROUP" ,"BANK OF AMERICA" ,"BANK OF BAHREIN AND KUWAIT" ,"BANK OF CEYLON", "BANK OF NOVA SCOTIA", "BARCLAYS BANK" ,"BNP PARIBAS BANK" ,"CITI BANK" ,"COMMONWEALTH BANK OF AUSTRALIA", "COOPERATIEVE RABOBANK U.A." ,"CREDIT AGRICOLE CORPORATE AND INVESTMENT BANK", "CREDIT SUISSE AG", "CTBC BANK CO., LTD" ,"DBS BANK INDIA LTD", "DEUTSCHE BANK" ,"FIRSTRAND BANK" ,"HONGKONG & SHANGHAI BANKING CORPORATION" ,"HSBC BANK OMAN S.A.O.G." ,"INDUSTRIAL AND COMMERCIAL BANK OF CHINA", "JP MORGAN CHASE BANK NA", "JSC VTB BANK", "KBC BANK NV" ,"KRUNGTHAI BANK" ,"MASHREQ BANK", "MIZUHO BANK", "MUFG BANK LTD", "NATIONAL AUSTRALIA BANK", "NatWest Markets Plc", "SBER BANK" ,"SBM BANK (INDIA) LTD" ,"SHINHAN BANK" ,"SOCIETE GENERALE" ,"SONALI BANK", "STANDARD CHARTERED BANK" ,"UBS AG" ,"UNITED OVERSEAS BANK" ,"WOORI BANK", "SUMITOMO MITSUI BANKING CORPORATION", "WESTPAC BANKING CORPORATION", "PT BANK MAYBANK INDONESIA TBK", "DOHA BANK QSC", "INDUSTRIAL BANK OF KOREA", "FIRST ABU DHABI BANK PJSC", "KEB HANA BANK", "Qatar National Bank SAQ", "EMIRATES NBD BANK", "BANK OF CHINA LTD", "KOOKMIN BANK"]
Private_Banks_3 = ["AXIS BANK", "CITY UNION BANK", "CATHOLIC SYRIAN BANK", "DCB BANK", "FEDERAL BANK", "HDFC BANK", "ICICI BANK", "IDBI", "INDUSIND BANK", "ING VYSYA BANK", "JAMMU AND KASHMIR BANK", "KARNATAKA BANK" ,"KARUR VYSYA BANK" ,"KOTAK MAHINDRA BANK", "LAXMI VILAS BANK" ,"NAINITAL BANK LTD", "RBL BANK", "SOUTH INDIAN BANK", "TAMILNADU MERCANTILE BANK", "DHANLAXMI BANK" ,"YES BANK", "IDFC FIRST BANK", "BANDHAN BANK"]
Public_Banks_3 = ["ALLAHABAD BANK", "ANDHRA BANK", "BANK OF BARODA", "BANK OF INDIA", "BANK OF MAHARASHTRA", "CANARA BANK", "CENTRAL BANK OF INDIA", "CORPORATION BANK", "DENA BANK", "IDBI", "INDIAN BANK", "INDIAN OVERSEAS BANK", "ORIENTAL BANK OF COMMERCE", "PUNJAB & SIND BANK", "PUNJAB NATIONAL BANK", "STATE BANK OF BIKANER AND JAIPUR", "STATE BANK OF HYDERABAD", "STATE BANK OF INDIA", "STATE BANK OF MYSORE", "STATE BANK OF PATIALA", "STATE BANK OF TRAVANCORE", "SYNDICATE BANK", "UCO BANK", "UNION BANK OF INDIA", "UNITED BANK OF INDIA", "VIJAYA BANK", "BHARATIYA MAHILA BANK"]
Small_Finance_3= ["CAPITAL SMALL FINANCE BANK", "EQUITAS SMALL FINANCE BANK", "ESAF SMALL FINANCE BANK", "FINCARE SMALL FINANCE BANK", "SURYODAY SMALL FINANCE BANK", "UJJIVAN SMALL FINANCE BANK", "AU SMALL FINANCE BANK", "NORTH EAST SMALL FINANCE BANK", "UTKARSH SMALL FINANCE BANK", "JANA SMALL FINANCE BANK"]
All_Banks_4 = Foreign_Banks_3 + Private_Banks_3 + Public_Banks_3 + Small_Finance_3
All_Banks_4 = [x.upper() for x in All_Banks_4]
All_Banks_4

['AB BANK',
 'ABU DHABI COMMERCIAL BANK',
 'AMERICAN EXPRESS BANKING CORP',
 'AUSTRALIA AND NEW ZEALAND BANKING GROUP',
 'BANK OF AMERICA',
 'BANK OF BAHREIN AND KUWAIT',
 'BANK OF CEYLON',
 'BANK OF NOVA SCOTIA',
 'BARCLAYS BANK',
 'BNP PARIBAS BANK',
 'CITI BANK',
 'COMMONWEALTH BANK OF AUSTRALIA',
 'COOPERATIEVE RABOBANK U.A.',
 'CREDIT AGRICOLE CORPORATE AND INVESTMENT BANK',
 'CREDIT SUISSE AG',
 'CTBC BANK CO., LTD',
 'DBS BANK INDIA LTD',
 'DEUTSCHE BANK',
 'FIRSTRAND BANK',
 'HONGKONG & SHANGHAI BANKING CORPORATION',
 'HSBC BANK OMAN S.A.O.G.',
 'INDUSTRIAL AND COMMERCIAL BANK OF CHINA',
 'JP MORGAN CHASE BANK NA',
 'JSC VTB BANK',
 'KBC BANK NV',
 'KRUNGTHAI BANK',
 'MASHREQ BANK',
 'MIZUHO BANK',
 'MUFG BANK LTD',
 'NATIONAL AUSTRALIA BANK',
 'NATWEST MARKETS PLC',
 'SBER BANK',
 'SBM BANK (INDIA) LTD',
 'SHINHAN BANK',
 'SOCIETE GENERALE',
 'SONALI BANK',
 'STANDARD CHARTERED BANK',
 'UBS AG',
 'UNITED OVERSEAS BANK',
 'WOORI BANK',
 'SUMITOMO MITSUI BANKING CORPORATION',
 '

In [46]:
inter_31 = []
for i in All_Banks_4:
    if i in branch_name_list_cap2:
        inter_31.append(i)
len(inter_31)

96

In [47]:
inter_41 = []
for i in branch_name_list_cap2:
    if i in All_Banks_4:
        inter_41.append(i)
len(inter_41)

98

In [48]:
bank_branch_code2['bank_name'] = bank_branch_code2['bank_name'].str.upper()
bank_branch_code2

,Code,bank_name
0,AACX,AKHAND ANAND CO.OP BANK
1,ABBL,AB BANK
2,ABCX,AURANGABAD DISTRICT CENTRAL CO-OPERATIVE BANK
3,ABDX,DR. AMBEDKAR NAGRIK SAHAKARI BANK MYDT GWALIOR
4,ABEX,ANDHRA BANK EMPLOYEES CO-OPERATIVE BANK
...,...,...
1426,ZSHX,ZILA SAHKARI BANK HARIDWAR
1427,ZSJX,ZILA SAHKARI BANK JHANSI
1428,ZSKX,ZILA SAHKARI BANK KANPUR
1429,ZSLX,ZILA SAHKARI BANK LAKHIMPUR KHERI


In [49]:
bank_branch_code_filter2 = bank_branch_code2[bank_branch_code2['bank_name'].isin(inter_31)]
bank_branch_code_filter2

,Code,bank_name
1,ABBL,AB BANK
18,ADCB,ABU DHABI COMMERCIAL BANK
44,ALLA,ALLAHABAD BANK
58,ANDB,ANDHRA BANK
63,ANZB,AUSTRALIA AND NEW ZEALAND BANKING GROUP
...,...,...
1338,UTKS,UTKARSH SMALL FINANCE BANK
1363,VIJB,VIJAYA BANK
1387,VYSA,ING VYSYA BANK
1397,WPAC,WESTPAC BANKING CORPORATION


In [50]:
bank_branch_code_filter2 = bank_branch_code_filter2[~bank_branch_code_filter2['Code'].isin(['ABBL', 'BMBL', 'CTBA', 'ESAF', 'FIRX', 'NATA', 'SBBJ', 'SBHY', 'SBLD', 'SBMY', 'SBTR', 'STBP', 'USFB', 'VYSA'])]

In [51]:
banks_to_read2 = bank_branch_code_filter2['Code']
banks_to_read2

18      ADCB
44      ALLA
58      ANDB
63      ANZB
85      AUBL
        ... 
1337    UTIB
1338    UTKS
1363    VIJB
1397    WPAC
1406    YESB
Name: Code, Length: 84, dtype: object

In [52]:
sorted(bank_branch_code_filter2['bank_name'].to_list())

['ABU DHABI COMMERCIAL BANK',
 'ALLAHABAD BANK',
 'ANDHRA BANK',
 'AU SMALL FINANCE BANK',
 'AUSTRALIA AND NEW ZEALAND BANKING GROUP',
 'AXIS BANK',
 'BANDHAN BANK',
 'BANK OF AMERICA',
 'BANK OF BAHREIN AND KUWAIT',
 'BANK OF BARODA',
 'BANK OF CEYLON',
 'BANK OF INDIA',
 'BANK OF MAHARASHTRA',
 'BANK OF NOVA SCOTIA',
 'BARCLAYS BANK',
 'BNP PARIBAS BANK',
 'CANARA BANK',
 'CAPITAL SMALL FINANCE BANK',
 'CATHOLIC SYRIAN BANK',
 'CENTRAL BANK OF INDIA',
 'CITI BANK',
 'CITY UNION BANK',
 'CORPORATION BANK',
 'CREDIT AGRICOLE CORPORATE AND INVESTMENT BANK',
 'CREDIT SUISSE AG',
 'DCB BANK',
 'DENA BANK',
 'DEUTSCHE BANK',
 'DHANLAXMI BANK',
 'DOHA BANK QSC',
 'EMIRATES NBD BANK',
 'EQUITAS SMALL FINANCE BANK',
 'ESAF SMALL FINANCE BANK',
 'FEDERAL BANK',
 'FINCARE SMALL FINANCE BANK',
 'FIRSTRAND BANK',
 'HDFC BANK',
 'HONGKONG & SHANGHAI BANKING CORPORATION',
 'ICICI BANK',
 'IDBI',
 'IDFC FIRST BANK',
 'INDIAN BANK',
 'INDIAN OVERSEAS BANK',
 'INDUSIND BANK',
 'INDUSTRIAL AND COMMERCI

In [55]:
data_dict2 = {}
for i in banks_to_read2:
    with open(f'Raw Data/by-bank_cov/by-bank 2/{i}.json', 'r') as file:
        data_dict2[i] = json.load(file)

In [56]:
len(data_dict2)

84

In [57]:
for i in banks_to_read2:
    globals()[f'bank_branch2_{i}'] = pd.json_normalize(data_dict2[i].values())

In [58]:
bank_branch2_HDFC

,IFSC,BRANCH,CENTRE,DISTRICT,STATE,ADDRESS,CONTACT,MICR,BANK,UPI,RTGS,CITY,NEFT,IMPS
0,HDFC0000001,TULSIANI CHMBRS - NARIMAN PT,MUMBAI,GREATER MUMBAI,MAHARASHTRA,101-104 TULSIANI CHAMBERSFREE PRESS JOURNAL MA...,9890603333,400240003,HDFC BANK,True,True,MUMBAI,True,True
1,HDFC0000002,MUMBAI - KHAR (WEST),MUMBAI,GREATER MUMBAI,MAHARASHTRA,"SWAGATAM,OPP. KHAR POLICE STATION,S.V.ROAD, KH...",9890603333,400240004,HDFC BANK,True,True,MUMBAI,True,True
2,HDFC0000003,K G MARG,DELHI,NEW DELHI,DELHI,"209 - 214, KAILASH BUILDING, 26, KASTURBA GAND...",61606161,110240001,HDFC BANK,True,True,DELHI,True,True
3,HDFC0000004,CHENNAI - ITC CENTRE - ANNA SALAI,CHENNAI,CHENNAI,TAMIL NADU,"759, ITC CENTREANNA SALAI, OPP T.V.S.CHENNAITA...",9840673333,600240002,HDFC BANK,True,True,CHENNAI,True,True
4,HDFC0000005,BHULABHAI DESAI ROAD,MUMBAI,GREATER MUMBAI,MAHARASHTRA,HDFC BANK LTD 36 VALLABH APTS 87 BHULABHAI DES...,61606161,400240005,HDFC BANK,True,True,MUMBAI,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6740,HDFC0CYACBP,Yashwant Co-operative Bank IMPS,KARAD,SATARA,MAHARASHTRA,"69,GURUWAR PETH,CHAWDI CHOWK,KARAD 415110",9422745818,415814352,Yashwant Co-operative Bank,True,True,KARAD,True,True
6741,HDFC0CYDCBL,YOUTH DEVELOPMENT COOP BANK LTD.KOL,KOLHAPUR,KOLHAPUR,MAHARASHTRA,"""POORNIMA"" F-2, FIRST FLOOR, 1429/G, NEAR SAMB...",2550120,416393015,HDFC BANK,True,True,KOLHAPUR,True,True
6742,HDFC0CYMCBK,Yeola mer co o pbank ltd,YEOLA,NASIK,MAHARASHTRA,Yashodhan Balaji Galli Yeola Dist Nasik,02559265047,423853352,HDFC BANK,True,True,YEOLA,True,True
6743,HDFC0CYMSB1,Yavatmal Mahila Sahakari Bank IMPS,YAVATMAL,YAVATMAL,MAHARASHTRA,"Annapurna, Mainde Chowk, Sankat Mochan Road Ya...",07232250661,445814002,Yavatmal Mahila Sahakari Bank,True,True,YAVATMAL,True,True


In [59]:
branch_all2 = pd.concat([globals()[f'bank_branch2_{i}'] for i in banks_to_read2], ignore_index=True)
branch_all2['foreign_dummy'] = branch_all2['BANK'].isin(Foreign_Banks_3).astype(int)
branch_all2['private_dummy'] = branch_all2['BANK'].isin(Private_Banks_3).astype(int)
branch_all2['public_dummy'] = branch_all2['BANK'].isin(Public_Banks_3).astype(int)
branch_all2['small_finance_dummy'] = branch_all2['BANK'].isin(Small_Finance_3).astype(int)

In [60]:
branch_all2

,BANK,IFSC,BRANCH,CENTRE,DISTRICT,STATE,ADDRESS,CONTACT,IMPS,CITY,UPI,RTGS,NEFT,MICR,foreign_dummy,private_dummy,public_dummy,small_finance_dummy
0,Abu Dhabi Commercial Bank,ADCB0000001,Abu Dhabi Commercial Bank IMPS,MUMBAI,GREATER BOMBAY,MAHARASHTRA,"75 Rehmat Manzil, Veer Nariman Road, Churchgat...",39534100,True,MUMBAI,False,True,True,None,0,0,0,0
1,ABU DHABI COMMERCIAL BANK,ADCB0000002,BANGALORE,NA,BANGALORE,KARNATAKA,"CITI CENTRE, 28, CHURCH STREET, OFF M. G. ROAD...",25582000,True,BANGALORE URBAN,False,False,True,None,1,0,0,0
2,ALLAHABAD BANK,ALLA0210001,Guwahati Main Branch,GUWAHATI,KAMRUP METROPOLITAN,ASSAM,"H. B. Road,Machkhowa, Opp. Of ASTC Bus Stand M...",2545218,True,GUWAHATI,True,True,True,None,0,0,1,0
3,ALLAHABAD BANK,ALLA0210002,PATNA UNIVERSITY,PATNA,PATNA,BIHAR,"PATNA UNIVERSITY.GANESH ROAD,DIST-PATNA",2671644,True,PATNA,True,True,True,None,0,0,1,0
4,ALLAHABAD BANK,ALLA0210003,PATNA MAIN,PATNA,PATNA,BIHAR,"BUDH MARG,DIST-PATNA,PIN-800001",2235237,True,PATNA,True,True,True,None,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138639,YES BANK,YESB0ZSBL20,ZILA SAHKARI BANK LUCKNOW ICCMRT EC,LUCKNOW,LUCKNOW,UTTAR PRADESH,"21/467, INDIRA NAGAR, LUCKNOW - 226016",8756400236,True,LUCKNOW,True,True,True,226185254,0,1,0,0
138640,YES BANK,YESB0ZSBL21,ZILA SAHKARI BANK LUCKNOW PARAGDARY,LUCKNOW,LUCKNOW,UTTAR PRADESH,"JOPPLING ROAD, LUCKNOW - 226001",8756400237,True,LUCKNOW,True,True,True,226185258,0,1,0,0
138641,YES BANK,YESB0ZSBL22,ZILA SAHKARI BANK LUCKNOW ARJUNGANJ,ARJUNGANJ,LUCKNOW,UTTAR PRADESH,"AHMAMAU, ARJUNGANJ, LUCKNOW - 226002",8756400239,True,ARJUNGANJ,True,True,True,226185252,0,1,0,0
138642,YES BANK,YESB0ZSBL23,ZILA SAHKARI BANK LUCKNOW CHANDRNGR,LUCKNOW,LUCKNOW,UTTAR PRADESH,"H NO 551 KA/180 KHA,ST BANK RD, BHILLAWAN,CHAN...",8756400238,True,LUCKNOW,True,True,True,226185257,0,1,0,0


In [ ]:
# #Uncomment to write to csv
# branch_all2.to_csv(f'Clean Data/branch_all_cov.csv')

In [62]:
def clean_state(df, x):
    if x == "agri":
        df_1 = df.iloc[~df.index.isin([0,1,2,3,38,39])]
        df_1.pop('Unnamed: 0')
        new_columns = [f"{df_1.iloc[0, i]}" for i in range(0,df_1.shape[1])]
        df_1.columns = new_columns
        col = df_1.columns
        col_2 = col[1:]
        col_3 = list(col_2)
        col_3.insert(0, 'state')
        df_1.columns = col_3
        df_2 = df_1.iloc[~df_1.index.isin([4])]
        df_2 = df_2.loc[:, ['state', '2014-15']]
        df_2.columns = ['state' , '2014-15_04', '2014-15_11']
        df_3 = df_2.loc[:, ['state', '2014-15_11']]
        df_3 = df_3.rename(columns={'2014-15_11':x})
        df_3[x] = df_3[x].replace('-', np.nan)
        df_3['state'] = df_3['state'].str.upper()
        return df_3
    elif x == "bank":
        df_1 = df.iloc[~df.index.isin([0,1,2,3,38,39,40])]
        df_1.pop('Unnamed: 0')
        new_columns = [f"{df_1.iloc[0, i]}" for i in range(0,df_1.shape[1])]
        df_1.columns = new_columns
        col = df_1.columns
        col_2 = col[1:]
        col_3 = list(col_2)
        col_3.insert(0, 'state')
        df_1.columns = col_3
        df_2 = df_1.iloc[~df_1.index.isin([4])]
        df_2 = df_2.loc[:, ['state', '2014-15']]
        df_2.columns = ['state' , '2014-15_04', '2014-15_11']
        df_3 = df_2.loc[:, ['state', '2014-15_11']]
        df_3 = df_3.rename(columns={'2014-15_11':x})
        df_3[x] = df_3[x].replace('-', np.nan)
        df_3['state'] = df_3['state'].str.upper()
        return df_3
    elif x == "const":
        df_1 = df.iloc[~df.index.isin([0,1,2,3,4,39,40])]
        df_1.pop('Unnamed: 0')
        new_columns = [f"{df_1.iloc[0, i]}" for i in range(0,df_1.shape[1])]
        df_1.columns = new_columns
        col = df_1.columns
        col_2 = col[1:]
        col_3 = list(col_2)
        col_3.insert(0, 'state')
        df_1.columns = col_3
        df_2 = df_1.iloc[~df_1.index.isin([5])]
        df_2 = df_2.loc[:, ['state', '2014-15']]
        df_2.columns = ['state' , '2014-15_04', '2014-15_11']
        df_3 = df_2.loc[:, ['state', '2014-15_11']]
        df_3 = df_3.rename(columns={'2014-15_11':x})
        df_3[x] = df_3[x].replace('-', np.nan)
        df_3['state'] = df_3['state'].str.upper()
        return df_3
    elif x == "ind":
        df_1 = df.iloc[~df.index.isin([0,1,2,3,4,39,40])]
        df_1.pop('Unnamed: 0')
        new_columns = [f"{df_1.iloc[0, i]}" for i in range(0,df_1.shape[1])]
        df_1.columns = new_columns
        col = df_1.columns
        col_2 = col[1:]
        col_3 = list(col_2)
        col_3.insert(0, 'state')
        df_1.columns = col_3
        df_2 = df_1.iloc[~df_1.index.isin([5])]
        df_2 = df_2.loc[:, ['state', '2014-15']]
        df_2.columns = ['state' , '2014-15_04', '2014-15_11']
        df_3 = df_2.loc[:, ['state', '2014-15_11']]
        df_3 = df_3.rename(columns={'2014-15_11':x})
        df_3[x] = df_3[x].replace('-', np.nan)
        df_3['state'] = df_3['state'].str.upper()
        return df_3
    elif x == "manu":
        df_1 = df.iloc[~df.index.isin([0,1,2,3,4,39,40,41])]
        df_1.pop('Unnamed: 0')
        new_columns = [f"{df_1.iloc[0, i]}" for i in range(0,df_1.shape[1])]
        df_1.columns = new_columns
        col = df_1.columns
        col_2 = col[1:]
        col_3 = list(col_2)
        col_3.insert(0, 'state')
        df_1.columns = col_3
        df_2 = df_1.iloc[~df_1.index.isin([5])]
        df_2 = df_2.loc[:, ['state', '2014-15']]
        df_2.columns = ['state' , '2014-15_04', '2014-15_11']
        df_3 = df_2.loc[:, ['state', '2014-15_11']]
        df_3 = df_3.rename(columns={'2014-15_11':x})
        df_3[x] = df_3[x].replace('-', np.nan)
        df_3['state'] = df_3['state'].str.upper()
        return df_3
    elif x == "serv":
        df_1 = df.iloc[~df.index.isin([0,1,2,3,4,39,40,41])]
        df_1.pop('Unnamed: 0')
        new_columns = [f"{df_1.iloc[0, i]}" for i in range(0,df_1.shape[1])]
        df_1.columns = new_columns
        col = df_1.columns
        col_2 = col[1:]
        col_3 = list(col_2)
        col_3.insert(0, 'state')
        df_1.columns = col_3
        df_2 = df_1.iloc[~df_1.index.isin([5])]
        df_2 = df_2.loc[:, ['state', '2014-15']]
        df_2.columns = ['state' , '2014-15_04', '2014-15_11']
        df_3 = df_2.loc[:, ['state', '2014-15_11']]
        df_3 = df_3.rename(columns={'2014-15_11':x})
        df_3[x] = df_3[x].replace('-', np.nan)
        df_3['state'] = df_3['state'].str.upper()
        return df_3

In [65]:
state_agri = clean_state(pd.read_excel('Raw Data/state_data/state_agri.XLSX'), "agri")
state_banking = clean_state(pd.read_excel('Raw Data/state_data/state_banking.XLSX'), "bank")
state_construction = clean_state(pd.read_excel('Raw Data/state_data/state_construction.XLSX'), "const")
state_industry = clean_state(pd.read_excel('Raw Data/state_data/state_industry.XLSX'), "ind")
state_manufacturing = clean_state(pd.read_excel('Raw Data/state_data/state_manufacturing.XLSX'), "manu")
state_services = clean_state(pd.read_excel('Raw Data/state_data/state_services.XLSX'), "serv")

In [66]:
state_vars = pd.merge(state_agri, state_banking, on = "state").merge(state_construction, on = "state").merge(state_industry, on = "state").merge(state_manufacturing, on = "state").merge(state_services, on = "state")

In [ ]:
# #Uncomment to write to csv
# state_vars.to_csv('Clean Data/state_vars.csv')